In [ ]:
import os
import re
import json
from pathlib import Path
from dotenv import load_dotenv
from tavily import TavilyClient
from serpapi import GoogleSearch
import openai

# ================================
# Load Environment Variables
# ================================

# looks for the .env file in the same directory as the script 
# The code automatically finds the right directory where the script 
# (or notebook) is running and works across  code editors: VS, Jupyter, and terminal.
#You can move the whole project folder anywhere, and it still works.
# It's cross-platform (Windows, macOS, Linux). No worries about slashes (\ vs /).
# Works inside VS Code, Jupyter, and the terminal the same way.
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()

# load_dotenv(...): Loads environment information from an .env. file
# that you will store in a variables from a .env file.
#script_dir / "SERPAPI_API_KEY.env": Builds the full path to SERPAPI_API_KEY.env 
#inside the script's directory.
load_dotenv(script_dir / "TAVILY_API_KEY.env")
load_dotenv(script_dir / "OPENAI_API_KEY.env")
load_dotenv(script_dir / "SERPAPI_API_KEY.env")

TAVILY_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_API_KEY")

if not TAVILY_KEY:
    raise ValueError("⚠️ TAVILY_API_KEY not found in environment file")
if not OPENAI_API_KEY:
    raise ValueError("⚠️ OPENAI_API_KEY not found in environment file")
if not SERPAPI_KEY:
    raise ValueError("⚠️ SERPAPI_API_KEY not found in environment file")

print(f"TAVILY_KEY loaded: {'Yes' if TAVILY_KEY else 'No'}")
print(f"OPENAI_API_KEY loaded: {'Yes' if OPENAI_API_KEY else 'No'}")
print(f"SERPAPI_KEY loaded: {'Yes' if SERPAPI_KEY else 'No'}")
print("=" * 60)

# ================================
# Initialize Clients
# ================================

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_KEY)

# Initialize OpenAI client
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)

# === Parameters ===
TRIP_START = "2025-11-10"
TRIP_END = "2025-11-20"
TRIP_NIGHTS = 10
TRIP_DAYS = 10

# === Helpers ===
def normalize_price(text):
    match = re.search(r"\$([\d,]+)", str(text))
    if match:
        return float(match.group(1).replace(",", ""))
    return None

def normalize_flight(result):
    return {
        "name": result.get("title"),
        "url": result.get("url"),
        "snippet": result.get("content"),
        "price": normalize_price(result.get("content", "")),
        "airline": None,
        "dates": {"start": TRIP_START, "end": TRIP_END}
    }

def normalize_hotel(result):
    nightly = normalize_price(result.get("content", ""))
    total = nightly * TRIP_NIGHTS if nightly else None
    return {
        "name": result.get("title"),
        "url": result.get("url"),
        "snippet": result.get("content"),
        "nightly_price": nightly,
        "total_price": total,
        "stars": None,
        "dates": {"start": TRIP_START, "end": TRIP_END}
    }

def normalize_car(result):
    daily = normalize_price(result.get("content", ""))
    total = daily * TRIP_DAYS if daily else None
    return {
        "company": result.get("title"),
        "url": result.get("url"),
        "snippet": result.get("content"),
        "daily_price": daily,
        "total_price": total,
        "dates": {"start": TRIP_START, "end": TRIP_END}
    }

def normalize_poi(result):
    return {
        "name": result.get("title"),
        "url": result.get("url"),
        "snippet": result.get("content")
    }

# --- Tavily Search ---
def tavily_search(query, normalizer, max_results=5):
    results = tavily_client.search(query, search_depth="advanced")
    return [normalizer(r) for r in results.get("results", [])[:max_results]]

# --- SerpAPI Wrappers ---
def serpapi_flights():
    search = GoogleSearch({
        "engine": "google_flights",
        "departure_id": "SFO",
        "arrival_id": "SJD",
        "outbound_date": TRIP_START,
        "return_date": TRIP_END,
        "api_key": SERPAPI_KEY
    })
    results = search.get_dict()
    items = []
    for flight in results.get("best_flights", [])[:5]:
        price = flight.get("price")
        items.append({
            "airline": flight.get("airline"),
            "price": price,
            "url": flight.get("link", "No link"),
            "dates": {"start": TRIP_START, "end": TRIP_END}
        })
    return items

def serpapi_hotels():
    search = GoogleSearch({
        "engine": "google_hotels",
        "q": "Hotels in Cabo San Lucas",
        "check_in_date": TRIP_START,
        "check_out_date": TRIP_END,
        "api_key": SERPAPI_KEY
    })
    results = search.get_dict()
    items = []
    for hotel in results.get("properties", [])[:5]:
        nightly = hotel.get("rate_per_night", {}).get("lowest")
        total = nightly * TRIP_NIGHTS if nightly else None
        items.append({
            "name": hotel.get("name"),
            "price_nightly": nightly,
            "price_total": total,
            "url": hotel.get("link", "No link")
        })
    return items

def serpapi_cars():
    search = GoogleSearch({
        "engine": "google",
        "q": "Cabo San Lucas car rentals",
        "api_key": SERPAPI_KEY
    })
    results = search.get_dict()
    items = []
    for r in results.get("organic_results", [])[:5]:
        daily = normalize_price(r.get("snippet", ""))
        total = daily * TRIP_DAYS if daily else None
        items.append({
            "company": r.get("title"),
            "url": r.get("link"),
            "snippet": r.get("snippet"),
            "daily_price": daily,
            "total_price": total,
            "dates": {"start": TRIP_START, "end": TRIP_END}
        })
    return items

# ================================
# GPT-4o-mini Trip Plan Generator
# ================================

def generate_trip_plan(trip_data):
    """
    Use GPT-4o-mini to summarize and format the search results into a comprehensive trip plan
    """
    
    # Create the prompt with search results
    prompt = f"""You are a world-class travel planner. Take the provided search results and format them
into a comprehensive, structured trip plan. Extract up to 3 flight options, even if they
come from different airlines or booking sites. If fewer than 3 are available, return as
many as possible. Do the same for hotels, cars, and POIs. Include prices, URLs, and
descriptions when available. If a piece of information is missing, leave that field as None.

Trip Details:
- Destination: Cabo San Lucas, Mexico
- Dates: {TRIP_START} to {TRIP_END} ({TRIP_NIGHTS} nights, {TRIP_DAYS} days)
- Departure: San Francisco (SFO)

Search Results Data:
{json.dumps(trip_data, indent=2, default=str)}

Please format your response as a well-structured trip plan with clear sections for:
1. FLIGHT OPTIONS (up to 3 best options)
2. HOTEL OPTIONS (up to 3 best options)
3. CAR RENTAL OPTIONS (up to 3 best options)
4. POINTS OF INTEREST (up to 3 top recommendations)

For each option, include all available information (prices, URLs, descriptions, etc.) and highlight the best value or recommended choices."""

    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=2000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating trip plan with GPT-4o-mini: {str(e)}"

# === Main ===
def main():
    print("🔍 Starting trip planning search...")
    trip_data = {}

    print("Searching flights with Tavily...")
    trip_data["flights_tavily"] = tavily_search(
        "Flights from San Francisco to Cabo San Lucas November 10 to November 20 2025 with prices",
        normalize_flight
    )
    
    print("Searching hotels with Tavily...")
    trip_data["hotels_tavily"] = tavily_search(
        "Hotels in Cabo San Lucas with prices in USD for November 10 to November 20 2025 and rating 4 star or higher",
        normalize_hotel
    )
    
    print("Searching car rentals with Tavily...")
    trip_data["cars_tavily"] = tavily_search(
        "Car rentals in Cabo San Lucas November 10 to November 20 2025 with prices",
        normalize_car
    )
    
    print("Searching points of interest with Tavily...")
    trip_data["pois_tavily"] = tavily_search(
        "Top points of interest in Cabo San Lucas",
        normalize_poi
    )

    print("Searching flights with SerpAPI...")
    trip_data["flights_serpapi"] = serpapi_flights()
    
    print("Searching hotels with SerpAPI...")
    trip_data["hotels_serpapi"] = serpapi_hotels()
    
    print("Searching car rentals with SerpAPI...")
    trip_data["cars_serpapi"] = serpapi_cars()

    # Save raw data to JSON file
    timestamp = TRIP_START.replace("-", "")  # Convert 2025-11-10 to 20251110
    json_filename = f"cabo_trip_data_{timestamp}.json"
    json_filepath = script_dir / json_filename
    
    try:
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(trip_data, f, indent=2, ensure_ascii=False, default=str)
        print(f"💾 Raw search data saved to: {json_filename}")
    except Exception as e:
        print(f"⚠️ Error saving JSON file: {e}")

    # # DEBUG: Let's see what data we actually collected
    # print("\n📊 DEBUG: Collected search results:")
    # print("-" * 40)
    # for key, value in trip_data.items():
    #     print(f"{key}: {len(value) if value else 0} results")
    #     if value and len(value) > 0:
    #         print(f"  Sample: {value[0]}")
    #     print()

    print("\n🤖 Generating trip plan with GPT-4o-mini...")
    
    # Generate final structured plan using GPT-4o-mini
    final_plan = generate_trip_plan(trip_data)

    print("\n" + "=" * 60)
    print("🌴 FINAL TRIP PLAN - CABO SAN LUCAS 🌴")
    print("=" * 60)
    print(final_plan)
    print("=" * 60)

if __name__ == "__main__":
    main()

TAVILY_KEY loaded: Yes
OPENAI_API_KEY loaded: Yes
SERPAPI_KEY loaded: Yes
🔍 Starting trip planning search...
Searching flights with Tavily...
Searching hotels with Tavily...
Searching car rentals with Tavily...
Searching points of interest with Tavily...
Searching flights with SerpAPI...
Searching hotels with SerpAPI...
Searching car rentals with SerpAPI...
💾 Raw search data saved to: cabo_trip_data_20251110.json

📊 DEBUG: Collected search results:
----------------------------------------
flights_tavily: 5 results
  Sample: {'name': 'Cheap flights from San Francisco to Cabo San Lucas - Google', 'url': 'https://www.google.com/travel/flights/flights-from-san-francisco-to-cabo-san-lucas.html?gl=US&hl=en-US', 'snippet': 'The cheapest Southwest flight from San Francisco to Cabo San Lucas is $315, flying from Nov 14 to Nov 20.\n The cheapest Aeromexico flight from San Francisco to Cabo San Lucas is $301, flying from Nov 13 to Nov 19. [...] The cheapest round-trip flight from San Francisco to